### Алина Шаймарданова

In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [2]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = '/Users/alinashaymardanova/Downloads/aot'

In [3]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [4]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [5]:
data.shape

(1987, 5)

In [6]:
data.head(3)

,content,keywords,summary,title,url
0,В среду состоялось отложенное заседание Совета...,"[школа, образовательные стандарты, литература,...","Глава Минобрнауки считает, что в нездоровом аж...","Ольга Васильева обещала ""НГ"" не перегружать шк...",https://amp.ng.ru/?p=http://www.ng.ru/educatio...
1,"Хорошо, когда красота в глазах смотрящего живе...","[красота, законы]",О живительной пользе укорота при выборе между ...,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/20...
2,Когда-то Леонид Юзефович написал книгу о монго...,"[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisima...


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [7]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [8]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [9]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [10]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [11]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [12]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [13]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0    [ольга, васильева, обещала, "нг", не, перегруж...
1            [у, красоты, собственные, закон, и, воля]
2                             [апокалиптический, бунт]
3    [f1., предсказать, результаты, гран-при, испан...
4                [возвращение, в, небесное, отечество]
5        [практическая, медицина, с, большим, будущим]
6    [бумажный, носитель., вселенная,, человек,, яз...
7       [в, багдаде, неспокойно,, а, будет, еще, хуже]
8    [ученые, предсказывают, появление, интуиции, у...
9    [россияне, от, вступления, в, вто, не, пострад...
Name: title, dtype: object

In [14]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                  [и, в, –, по, на, что, с, о, к, не]
1       [и, в, не, что, –, на, красота, вы, если, как]
2           [в, и, не, на, –, от, о, из, пепеляев, но]
3              [в, не, и, на, что, но, это, как, с, к]
4                [и, в, –, не, он, с, на, что, а, его]
5             [и, в, на, –, с, что, не, это, как, уже]
6                 [–, и, в, не, на, из, по, что, с, а]
7             [в, и, не, на, но, ли, ираке, что, -, а]
8                 [и, в, на, о, с, –, как, а, что, ai]
9    [в, на, по, и, что, кристалина, –, россии, гео...
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [15]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [16]:
data['content_norm'] = data['content'].apply(normalize)

In [17]:
data['title_norm'] = data['title'].apply(normalize)

In [18]:
data['title_norm'].head(10)

0    [ольга, васильев, обещать, нг, перегружать, шк...
1                  [красота, собственный, закон, воля]
2                             [апокалиптический, бунт]
3    [f1, предсказать, результат, гран-при, испания...
4                   [возвращение, небесный, отечество]
5           [практический, медицина, больший, будущий]
6    [бумажный, носитель, вселенная, человек, язык,...
7                          [багдад, неспокойно, худой]
8    [учёный, предсказывать, появление, интуиция, и...
9             [россиянин, вступление, вто, пострадать]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [19]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [20]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [21]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [стандарт, который, источник, образовательный,...
1     [красота, ваш, глаз, отчаяние, уйти, это, поры...
2     [пепеляев, юзеф, книга, якутия, восстание, год...
3     [гонка, команда, это, сказать, пилот, ferrari,...
4     [есенин, поэт, клюев, год, свой, это, жизнь, к...
5     [наш, медицина, медицинский, это, кафедра, вып...
6     [книга, русский, человек, мозг, два, островной...
7     [ирак, война, американец, это, партизанский, в...
8     [который, нейросеть, свой, ai, клетка, искусст...
9     [россия, вступление, кристалина, георгиев, вто...
10    [фильм, приз, который, картина, свой, кино, на...
11    [это, каша, сидеть, столовый, который, человек...
12    [место, псков, город, день, такой, князь, андр...
13    [индонезия, который, страна, год, мусульмански...
14    [это, свой, такой, который, игра, каждый, друг...
15    [выбор, путин, президент, признавать, сша, рф,...
16    [водный, год, вода, качество, загрязнение, объ...
17    [школа, образование, год, москва, школьник

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [22]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [23]:
data['content_norm'] = data['content'].apply(normalize)

In [25]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.1


Ещу улучшения!

In [26]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0    [стандарт, источник, фгоса, ольга, васильев, р...
1    [красота, глаз, отчаяние, порыв, кошка, предме...
2    [пепеляев, юзеф, книга, якутия, восстание, год...
3    [гонка, команда, пилот, сезон, время, машина, ...
4    [есенин, поэт, клюев, год, жизнь, смерть, серг...
5    [медицина, кафедра, выпускник, уровень, работа...
6    [книга, человек, мозг, говор, анатомия, глава,...
7    [ирак, война, американец, войско, структура, п...
8    [нейросеть, клетка, интеллект, время, основа, ...
9    [россия, вступление, кристалина, георгиев, вто...
Name: content_norm, dtype: object

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [27]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [28]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [29]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [30]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [31]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [32]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [33]:
keywords[:3]

[['ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'произведение',
  'письмо',
  'редакция',
  'содержание'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'руина',
  'предмет',
  'нелепость',
  'занавеска',
  'мечта'],
 ['пепеляев',
  'юзеф',
  'якутия',
  'восстание',
  'книга',
  'леонид юзеф',
  'сибирь',
  'леонид',
  '20 ха',
  'красных']]

In [34]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [35]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [36]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [37]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 1min 53s, sys: 984 ms, total: 1min 54s
Wall time: 1min 56s


In [38]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [39]:
keywords_rw.head(10)

0                     [источник, васильев, стандарт, фгоса, письмо, ольга, содержание, автор, предмет, произведение]
1                               [красота, отчаяние, руина, глаз, порыв, предмет, место, нелепость, мечта, занавеска]
2                                  [пепеляев, юзеф, якутия, кампания, книга, армия, мороз, фронтовик, обиняк, место]
3                                   [команда, гонка, сезон, пилот, разница, факт, трасса, время, себастьян, уик-энд]
4                               [есенин, поэт, жизнь, пружина, сергей, декабрь, революция, год, сереженька, николай]
5                          [медицина, кафедра, выпускник, уровень, рудна, год, врач, гражданин, практика, ординатор]
6                   [книга, человек, анатомия, учебник, группа, говор, азербайджан, исследование, лексикон, эльвира]
7                                [ирак, американец, война, оружие, количество, вертолёт, день, власть, абрамс, танк]
8    [человек, нейросеть, успех, изображение, вычисление, интелл

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [38]:
import networkx as nx

In [37]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [42]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 28.7 s, sys: 417 ms, total: 29.1 s
Wall time: 30.1 s


In [43]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другой.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 0.5 баллов).

#### А если анализировать и заголовки тоже? Определение ключевых слов текста по заголовку, как рассказывали нам в прошлогоднем курсе по автоматической обработке естественого языка, тоже является весьма эффекстивным способом.

In [39]:
data['content_norm_title'] = (data['content'] + data['title']).apply(normalize).apply(' '.join)

In [40]:
tfidf_title = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf_title.fit(data['content_norm_title'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [41]:
id2word_title = {i:word for i, word in enumerate(tfidf_title.get_feature_names())}

In [42]:
texts_vectors_title = tfidf_title.transform(data['content_norm_title'])

In [43]:
evaluate(data['keywords'], [[id2word_title[w] for w in top] for top in texts_vectors_title.toarray().argsort()[:,:-11:-1]])

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


#### Уже хорошо. Теперь попробуем использовать mystem на материале без заголовков.

In [44]:
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
import string
mystem = Mystem()
punct = punctuation + '«»—…“”*№–'

In [45]:
def normalize_mystem(text):
    words = []
    analysis = mystem.analyze(text)
    for el in analysis:
        try:
            lexeme = el['analysis'][0]['lex']
            if el['analysis'][0]['gr'].split(',')[0] == 'S'\
            and lexeme not in stops:
                words.append(lexeme)
        except:
            continue

    return words

In [46]:
data['content_norm_ms'] = data['content'].apply(normalize_mystem).apply(' '.join)

In [47]:
tfidf_ms = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf_ms.fit(data['content_norm_ms'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [48]:
id2word_ms = {i:word for i, word in enumerate(tfidf_ms.get_feature_names())}

In [49]:
texts_vectors_ms = tfidf_ms.transform(data['content_norm_ms'])

In [50]:
evaluate(data['keywords'], [[id2word_ms[w] for w in top] for top in texts_vectors_ms.toarray().argsort()[:,:-11:-1]])

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


#### А если вывести не топ 10, а топ 5?

+ (mystem на материале без заголовков)

In [51]:
evaluate(data['keywords'], [[id2word_ms[w] for w in top] for top in texts_vectors_ms.toarray().argsort()[:,:-6:-1]])

Precision -  0.2
Recall -  0.19
F1 -  0.19
Jaccard -  0.11


+ (pymorphy на материале с заголовками)

In [52]:
evaluate(data['keywords'], [[id2word_title[w] for w in top] for top in texts_vectors_title.toarray().argsort()[:,:-6:-1]])

Precision -  0.2
Recall -  0.19
F1 -  0.18
Jaccard -  0.11


#### В общем, mystem работает лучше, это видно. Нужно попробовать mystem и заголовки.

In [53]:
data['content_norm_ms_title'] = (data['content'] + data['title']).apply(normalize_mystem).apply(' '.join)

In [54]:
tfidf_ms = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf_ms.fit(data['content_norm_ms_title'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [55]:
id2word_ms = {i:word for i, word in enumerate(tfidf_ms.get_feature_names())}

In [56]:
texts_vectors_ms = tfidf_ms.transform(data['content_norm_ms_title'])

+ топ 10

In [57]:
evaluate(data['keywords'], [[id2word_ms[w] for w in top] for top in texts_vectors_ms.toarray().argsort()[:,:-11:-1]])

Precision -  0.14
Recall -  0.27
F1 -  0.18
Jaccard -  0.1


+ топ 5

In [58]:
evaluate(data['keywords'], [[id2word_ms[w] for w in top] for top in texts_vectors_ms.toarray().argsort()[:,:-6:-1]])

Precision -  0.21
Recall -  0.2
F1 -  0.2
Jaccard -  0.12


#### А если провернуть махинации с топ 5/10 на графе?

+ топ 10

In [59]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 25 s, sys: 326 ms, total: 25.3 s
Wall time: 25.5 s


In [60]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


+ топ 5

In [61]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))

CPU times: user 25.5 s, sys: 324 ms, total: 25.9 s
Wall time: 26.1 s


In [62]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.17
Recall -  0.17
F1 -  0.16
Jaccard -  0.1


Почему-то стало только хуже. Но, попытка не пытка:)